In [1]:
import os

%pwd

os.chdir("../")


%pwd


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
 
from cnnClassifier.constants import *
from cnnClassifier.utils.comon import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        print(f"Loading parameters from: {params_filepath}")




    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


import os
import zipfile
import gdown
from cnnClassifier.utils import logger
from cnnClassifier.utils.comon import get_size




class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e



[2024-09-16 11:23:09,043: INFO: comon: yaml file: config\config.yaml loaded successfully]
[2024-09-16 11:23:09,047: INFO: comon: yaml file: params.yaml loaded successfully]
[2024-09-16 11:23:09,049: INFO: comon: created directory at: artifacts]
Loading parameters from: params.yaml
[2024-09-16 11:23:09,052: INFO: comon: created directory at: artifacts/data_ingestion]
[2024-09-16 11:23:09,054: INFO: 1518292854: Downloading data from https://drive.google.com/file/d/1OBpczN6HAor-6r1rOQhTUATM59JSPCpp/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1OBpczN6HAor-6r1rOQhTUATM59JSPCpp
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1OBpczN6HAor-6r1rOQhTUATM59JSPCpp&confirm=t&uuid=86abccfa-d6e9-4a2d-90ad-2cdb8e8a1b34
To: c:\Users\shuda\End-to-end-with-MLFLOW\artifacts\data_ingestion\data.zip
100%|██████████| 110M/110M [00:37<00:00, 2.97MB/s] 

[2024-09-16 11:23:50,913: INFO: 1518292854: Downloaded data from https://drive.google.com/file/d/1OBpczN6HAor-6r1rOQhTUATM59JSPCpp/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


: 